In [1]:
import datetime
import re

from bs4 import BeautifulSoup
import requests

In [2]:
def get_category_urls(category_url,i=''):
    result = []
    page_url = category_url+'&p='+str(i)
    while True:
        print(page_url)
        soup = BeautifulSoup(requests.get(page_url).text,'html.parser')
        urls = ['http://www.setn.com'+soup.select("ol a")[i].attrs['href'] for i in range(len(soup.select("ol a")))]
        result.extend(urls)
        next_link = soup.find_all("div", attrs={'class' : 'pager'})[0].find_all('a')[-1]['href']
        if page_url=='http://www.setn.com'+next_link:
            break
            
        page_url = 'http://www.setn.com'+next_link
    
    return result

In [3]:
url = 'http://www.setn.com/ViewAll.aspx?pagegroupid=5'
urls = get_category_urls(url,'70')
len(urls)

http://www.setn.com/ViewAll.aspx?pagegroupid=5&p=70
http://www.setn.com/ViewAll.aspx?pagegroupid=5&p=71
http://www.setn.com/ViewAll.aspx?pagegroupid=5&p=72
http://www.setn.com/ViewAll.aspx?pagegroupid=5&p=73


93

In [4]:
def parser_page(url):
    """
    parser page method 
    主要用於抓取新聞detail頁面資訊

    page_data = parser_page(news detail url)
    """
    soup = BeautifulSoup(requests.get(url).text,'html.parser')
    search = lambda typ,seed: soup.find_all("meta", attrs={typ : seed})[0].attrs['content']
    inputs = [("property","og:url"),("property","og:title"),("name","pubdate"),("name","Keywords"),("name","section")]
    
    parse = [search(typ,seed) for typ,seed in inputs]
    post_time = datetime.datetime(*map(lambda x: int(x),re.findall('[0-9]+',parse[2])))
    
    article = soup.select("article div > p")
    
    content = []; journalist = None
    for i in range(len(article)):
        if len(article[i].attrs) == 0 and not journalist:
            journalist = article[i].text
        else:
            content.append(article[i].text)   
    content = "".join(content)
    
    fb_url = 'https://graph.facebook.com/fql?q=SELECT%20like_count,%20total_count,%20share_count, \
              %20click_count,%20comment_count%20FROM%20link_stat%20WHERE%20url%20=%20%22'+parse[0]+'%22'
    
    res_data = requests.get(fb_url).json()['data'][0]
    fb_like = res_data['like_count']
    fb_share = res_data['share_count']
    fb_com = res_data['comment_count']
    
    ######facebook comments#######   
    tree = {}
    fb_com_url = 'http://graph.facebook.com/comments?id=' + parse[0]+ \
                 '&limit=100&filter=stream&fields=parent.fields%28id%29,message,from,like_count,created_time,parent'
    fb_data = requests.get(fb_com_url).json()
    while True:
        if len(fb_data["data"])==0:
            break
        for comment in fb_data['data']:
            one_com = {}
            one_com["post_time"] = datetime.datetime(*map(lambda x: int(x),re.findall('[0-9]+',comment['created_time'][:-5])))
            one_com["actor"] = comment['from']['name']
            one_com["like"] = comment['like_count']
            one_com["content"] = comment['message']
            one_com["source_type"] = "facebook"
            one_com["dislike"] = None
            if 'parent' in comment:
                tree[comment['parent']['id']]["sub_comments"].append(one_com)
            else:
                one_com["sub_comments"] = []
                tree[comment['id']] = one_com

        fb_next_url = fb_com_url+'&after='+fb_data['paging']['cursors']['after']
        fb_data = requests.get(fb_next_url).json()
    ######facebook comments#######        
    
    
    return {
        "url": parse[0],
        "source_press": None,
        "title": parse[1],
        "post_time": post_time,
        "journalist": journalist,
        "content": content,
        "compare": None,
        "keyword": parse[3].split(','),
        "fb_like": fb_like,
        "fb_share": fb_share,
        "fb_comment_count": fb_com,
        "category": parse[4].split(','),
        "comment": list(tree.values())
    }

In [5]:
import numpy as np
parse_content = []
for i in np.random.choice(len(urls), 10,replace=False):
    print(i)
    parse_content.append(parser_page(urls[i]))
parse_content

34
2
19
92
35
42
8
81
33
58


[{'category': ['國際'],
  'comment': [{'actor': '陳雪雪',
    'content': '人生何其長 哪能只上一張床 +賴:4488by',
    'dislike': None,
    'like': 0,
    'post_time': datetime.datetime(2015, 10, 17, 10, 42, 36),
    'source_type': 'facebook',
    'sub_comments': []}],
  'compare': None,
  'content': '行之有年的愛爾蘭比基尼小姐大賽年年都吸引目光、殺光底片，今年特地搭上「國際乳癌防治月」，邀請一群身材姣好的模特兒，在首都都柏林（Dublin）街頭拍「上空」辣照。不過這次受盡矚目的卻不只是模特兒婀娜的姿態，而是一張令人疑惑、哭笑不得的照片。▲圖／翻攝自iefimerida\xa0這張由帕維爾（Ana Pavel）拍下的美照，在網路上瘋傳。她腳踩黑色高跟鞋、身穿黑色丁字褲，屁股蛋大喇喇對著鏡頭，一手拿著脫下來的內衣、一手遮著豐滿的胸部，嫵媚回眸讓人快噴鼻血！但這張照片被瘋傳的原因卻走偏了，仔細一看，前方一輛電車正筆直衝過來，女模與電車中間還夾著一名單車騎士，就這麼剛剛好跌倒在地，滑稽的鏡頭正好被拍下來。▲圖／翻攝自Twitter\xa0有網友稱讚「不管從任何角度來看，這都是是百分之百最棒的照片！」也有人猜測，那名騎單車的人可能就是看到模特兒的曼妙身材，才會分神而跌倒的。還有人發揮創意，模仿辣模想拍出一樣的照片，找來頂著啤酒肚的大男人站在軌道邊，身穿著黑色小短褲、手中抓著黑色內衣，背景也有一位跌倒的單車騎士，模仿得唯妙唯肖。（整理：實習編輯何昭融）▲圖／翻攝自Twitter\xa0》給我們一個讚，即時關心更多重大國際新聞》》 ',
  'fb_comment_count': 0,
  'fb_like': 21,
  'fb_share': 5,
  'journalist': '國際中心／綜合報導',
  'keyword': ['比基尼',
   '模特兒',
   '丁字褲',
   '乳癌',
   '愛爾蘭',
   '上空照',
   '三立',
   '三立新聞',
   '三立新聞台',
   '三

In [ ]:
'http://www.setn.com/News.aspx?NewsID=117306&PageGroupID=6&ProjectID=987'
'http://www.setn.com/News.aspx?NewsID=117355&PageGroupID=4&ProjectID=1037'
'http://www.setn.com/News.aspx?NewsID=118782'

In [161]:
def handleDefines(x):
    return x

#eval(re.search('handleDefines\(\[\[\"PlatformVersions".*\["UserActionHistory"\]\]}\)',resp.text).group())

In [7]:
example = 'http://www.setn.com/News.aspx?NewsID=117306&PageGroupID=6&ProjectID=987'
tree = {}
fb_com_url = 'http://graph.facebook.com/comments?id=http://www.setn.com/News.aspx?NewsID='+ \
              re.findall('NewsID=([0-9]+)',example)[0]+ \
             '&limit=100&filter=stream&fields=parent.fields%28id%29,message,from,like_count,created_time,parent'


fb_data = requests.get(fb_com_url).json()
while True:
    if len(fb_data["data"])==0:
        break
    for comment in fb_data['data']:
        one_com = {}
        one_com["post_time"] = datetime.datetime(*map(lambda x: int(x),re.findall('[0-9]+',comment['created_time'][:-5])))
        one_com["actor"] = comment['from']['name']
        one_com["like"] = comment['like_count']
        one_com["content"] = comment['message']
        one_com["source_type"] = "facebook"
        one_com["dislike"] = None
        if 'parent' in comment:
            tree[comment['parent']['id']]["sub_comments"].append(one_com)
        else:
            one_com["sub_comments"] = []
            tree[comment['id']] = one_com
    
    fb_next_url = fb_com_url+'&after='+fb_data['paging']['cursors']['after']
    fb_data = requests.get(fb_next_url).json()
    

In [8]:
tree

{'1191274760900590_1191374734223926': {'actor': '王又將',
  'content': '柯P加油！',
  'dislike': None,
  'like': 4,
  'post_time': datetime.datetime(2016, 1, 8, 20, 19, 26),
  'source_type': 'facebook',
  'sub_comments': []},
 '1191274760900590_1191607537533979': {'actor': 'Sam Wen',
  'content': '阿伯超強',
  'dislike': None,
  'like': 1,
  'post_time': datetime.datetime(2016, 1, 9, 6, 25),
  'source_type': 'facebook',
  'sub_comments': []},
 '1191274760900590_1191816480846418': {'actor': '張夢雅',
  'content': '【僅.限.臺.灣】完美的夢中情人與你相約+L.INE:tea555sexy\n\n【看.主.頁約.妹】 性.福.S.kpye:mmlove.0510\n\n【見本人滿意再現金消.費喔】 \n\n★外送地區：【台北】【台中】【高雄】【新竹】【彰化】',
  'dislike': None,
  'like': 0,
  'post_time': datetime.datetime(2016, 1, 9, 14, 56, 25),
  'source_type': 'facebook',
  'sub_comments': []},
 '1191274760900590_1191838790844187': {'actor': 'Lan Ping Li',
  'content': '一路好走',
  'dislike': None,
  'like': 0,
  'post_time': datetime.datetime(2016, 1, 9, 15, 36, 4),
  'source_type': 'facebook',
  'sub_comments': []},
 '